In [93]:
!pip install tensorflow_recommenders

In [94]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import tensorflow_recommenders as tfrs

In [95]:
from random import randint

In [96]:
ratingDf = pd.read_csv('../input/rating-data/rating_data.csv',usecols = ['iid','uid','rating'])

In [97]:
r,c = ratingDf.shape

In [98]:
emotions = [randint(1,7) for i in range(r)]
ratingDf['emotions'] = emotions

In [99]:
ratingDf['iid'] = ratingDf['iid'].astype(str)
ratingDf['uid'] = ratingDf['uid'].astype(str)
ratingDf['emotions'] = ratingDf['uid'].astype(str)

In [100]:
ratings = {name: np.array(value) for name, value in ratingDf.items()}

In [101]:
ratingDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21424 entries, 0 to 21423
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   uid       21424 non-null  object
 1   iid       21424 non-null  object
 2   rating    21424 non-null  int64 
 3   emotions  21424 non-null  object
dtypes: int64(1), object(3)
memory usage: 669.6+ KB


In [102]:
unique_user_ids = ratingDf['uid'].unique().astype(str)
unique_music_ids = ratingDf['iid'].unique().astype(str)
unique_emotions = ratingDf['emotions'].unique().astype(str)

In [103]:
class RankingModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        embedding_dimension = 32
        # Compute embeddings for users.
        self.user_embeddings = tf.keras.Sequential([
          tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
          tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
        ])
        
        # Compute embeddings for items.
        self.music_embeddings = tf.keras.Sequential([
          tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_music_ids, mask_token=None),
          tf.keras.layers.Embedding(len(unique_doctor_ids) + 1, embedding_dimension)
        ])
        
        self.emtion_embeddings = tf.keras.Sequential([
          tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_emotions, mask_token=None),
          tf.keras.layers.Embedding(len(unique_emotions) + 1, embedding_dimension)
        ])

        # Compute predictions.
        self.ratings = tf.keras.Sequential([
          # Learn multiple dense layers.
           tf.keras.layers.Dense(256, activation="relu"),
           tf.keras.layers.Dense(64, activation="relu"),
          # Make rating predictions in the final layer.
          tf.keras.layers.Dense(1)
      ])

    def call(self, inputs):
        user_id, doctor_id ,emotions = inputs

        user_embedding = self.user_embeddings(user_id)
        music_embedding = self.music_embeddings(doctor_id)
        emotion_embeddings = self.emtion_embeddings(emotions)
        
        return self.ratings(tf.concat([user_embedding, music_embedding,emotion_embeddings], axis=1))

    
class MusicRecommenderModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
          loss = tf.keras.losses.MeanSquaredError(),
          metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        features = features[0]
        rating_predictions = self.ranking_model(
        (features["uid"], features["iid"],features["emotions"]))

    # The task computes the loss and the metrics.
        return self.task(labels=features["rating"], predictions=rating_predictions)

# making the model

model = MusicRecommenderModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# fitting the model on ratings dataset

model.fit(ratings, epochs = 20,verbose = True)

# saving the model

Epoch 1/20
670/670 [==============================] - 3s 3ms/step - root_mean_squared_error: 1.0387 - loss: 1.0784 - regularization_loss: 0.0000e+00 - total_loss: 1.0784
Epoch 2/20
670/670 [==============================] - 2s 3ms/step - root_mean_squared_error: 1.0030 - loss: 1.0061 - regularization_loss: 0.0000e+00 - total_loss: 1.0061
Epoch 3/20
670/670 [==============================] - 2s 3ms/step - root_mean_squared_error: 0.9869 - loss: 0.9735 - regularization_loss: 0.0000e+00 - total_loss: 0.9735
Epoch 4/20
670/670 [==============================] - 2s 2ms/step - root_mean_squared_error: 0.9741 - loss: 0.9496 - regularization_loss: 0.0000e+00 - total_loss: 0.9496
Epoch 5/20
670/670 [==============================] - 2s 2ms/step - root_mean_squared_error: 0.9602 - loss: 0.9222 - regularization_loss: 0.0000e+00 - total_loss: 0.9222
Epoch 6/20
670/670 [==============================] - 2s 2ms/step - root_mean_squared_error: 0.9416 - loss: 0.8869 - regularization_loss: 0.0000e+00 -

In [104]:
model.ranking_model.predict((np.array(['190827.0']),np.array(['4941.0']),np.array(['7'])))

array([[2.8645241]], dtype=float32)

In [105]:
model.ranking_model.save('my_model')

In [109]:
model_loaded = tf.keras.models.load_model('./my_model')

In [110]:
model_loaded((np.array(['190827.0']),np.array(['4941.0']),np.array(['7'])))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.8645241]], dtype=float32)>